# Artificial Neural Networks and Deep Learning

---

## Homework 2: Minimal Working Example

To make your first submission, follow these steps:
1. Create a folder named `[2024-2025] AN2DL/Homework 2` in your Google Drive.
2. Upload the `mars_for_students.npz` file to this folder.
3. Upload the Jupyter notebook `Homework 2 - Minimal Working Example.ipynb`.
4. Load and process the data.
5. Implement and train your model.
6. Submit the generated `.csv` file to Kaggle.


## 🌐 Connect Colab to Google Drive

In [28]:
from google.colab import drive

drive.mount("/gdrive")
# Change if necessary
%cd /gdrive/My Drive/Uni/Magistrale/Poli/Artificial Neural Networks and Deep Learning/challenges/challenge2

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/Uni/Magistrale/Poli/Artificial Neural Networks and Deep Learning/challenges/challenge2


## ⚙️ Import Libraries

In [29]:
import os
from datetime import datetime

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras import backend as K

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import scipy

import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {tfk.__version__}")
print(f"GPU devices: {len(tf.config.list_physical_devices('GPU'))}")

TensorFlow version: 2.17.1
Keras version: 3.5.0
GPU devices: 1


## ⏳ Load the Data

In [30]:
data = np.load("mars_for_students.npz")

training_set = data["training_set"]
X_train = training_set[:, 0]
y_train = training_set[:, 1]

X_test = data["test_set"]

# Separiamo il train set in training e validation set
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)



input_shape = X_train.shape[1:]
num_classes = len(np.unique(y_train))



# Add color channel and rescale pixels between 0 and 1
X_train = X_train[..., np.newaxis] / 255.0
###
X_val = X_val[..., np.newaxis] / 255.0
X_test = X_test[..., np.newaxis] / 255.0

input_shape = X_train.shape[1:]
num_classes = len(np.unique(y_train))


In [31]:
print(f"Training X shape: {X_train.shape}")
print(f"Training y shape: {y_train.shape}")
print(f"Validation X shape: {X_val.shape}")
print(f"Validation y shape: {y_val.shape}")
print(f"Test X shape: {X_test.shape}")
print(f"Input shape: {input_shape}")
print(f"Number of classes: {num_classes}")


Training X shape: (2092, 64, 128, 1)
Training y shape: (2092, 64, 128)
Validation X shape: (523, 64, 128, 1)
Validation y shape: (523, 64, 128)
Test X shape: (10022, 64, 128, 1)
Input shape: (64, 128, 1)
Number of classes: 5


## 🛠️ Train and Save the Model

In [33]:
def unet(input_shape, num_classes):

    def residual_block(input_tensor, filters, kernel_size=3):
        x = tf.keras.layers.Conv2D(filters, kernel_size, padding="same")(input_tensor)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ReLU()(x)

        x = tf.keras.layers.Conv2D(filters, kernel_size, padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)

        if input_tensor.shape[-1] != filters: # To avoid different shapes
            input_tensor = tf.keras.layers.Conv2D(filters, (1, 1), padding="same")(input_tensor)

        x = tf.keras.layers.Add()([x, input_tensor])
        x = tf.keras.layers.ReLU()(x)
        return x

    def bottleneck_layer(input_tensor, filters):
        x = tf.keras.layers.Conv2D(filters // 2, 1, padding="same")(input_tensor)
        x = tf.keras.layers.ReLU()(x)

        x = tf.keras.layers.Conv2D(filters, 3, padding="same")(x)
        x = tf.keras.layers.ReLU()(x)

        x = tf.keras.layers.Conv2D(filters // 2, 1, padding="same")(x)
        return x

    def downsample_block(input_tensor, filters):
        x = residual_block(input_tensor, filters)
        p = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding="same")(x)
        return x, p

    def upsample_block(input_tensor, skip_tensor, filters):
        x = tf.keras.layers.Conv2DTranspose(filters, 3, strides=(2, 2), padding="same")(input_tensor)
        x = tf.keras.layers.Concatenate()([x, skip_tensor])
        x = residual_block(x, filters)
        return x

    inputs = tf.keras.layers.Input(input_shape)

    # Encoder
    s1, p1 = downsample_block(inputs, 64)
    s2, p2 = downsample_block(p1, 128)
    s3, p3 = downsample_block(p2, 256)

    b = bottleneck_layer(p3, 512)

    # Decoder
    d3 = upsample_block(b, s3, 256)
    d2 = upsample_block(d3, s2, 128)
    d1 = upsample_block(d2, s1, 64)

    outputs = tf.keras.layers.Conv2D(num_classes, 1, activation="softmax")(d1)

    return tf.keras.Model(inputs, outputs)

In [34]:
model = unet(input_shape=(64, 128, 1), num_classes=5)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 64, 128, 1)     │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_12 (Conv2D)        │ (None, 64, 128, 64)    │            640 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_12    │ (None, 64, 128, 64)    │            256 │ conv2d_12[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_9 (ReLU)            │ (None, 64, 128, 64)    │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_13 (Conv2D)        │ (None, 64, 128, 64)    │         36,928 │ re_lu_9[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_13    │ (None, 64, 128, 64)    │            256 │ conv2d_13[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_14 (Conv2D)        │ (None, 64, 128, 64)    │            128 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_6 (Add)               │ (None, 64, 128, 64)    │              0 │ batch_normalization_1… │
│                           │                        │                │ conv2d_14[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_10 (ReLU)           │ (None, 64, 128, 64)    │              0 │ add_6[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_3           │ (None, 32, 64, 64)     │              0 │ re_lu_10[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_15 (Conv2D)        │ (None, 32, 64, 128)    │         73,856 │ max_pooling2d_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_14    │ (None, 32, 64, 128)    │            512 │ conv2d_15[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_11 (ReLU)           │ (None, 32, 64, 128)    │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_16 (Conv2D)        │ (None, 32, 64, 128)    │        147,584 │ re_lu_11[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_15    │ (None, 32, 64, 128)    │            512 │ conv2d_16[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_17 (Conv2D)        │ (None, 32, 64, 128)    │          8,320 │ max_pooling2d_3[0][0]  │
├──────────────────────

 Total params: 6,025,221 (22.98 MB)

 Trainable params: 6,021,637 (22.97 MB)

 Non-trainable params: 3,584 (14.00 KB)

In [35]:
# Define the MeanIoU ignoring the background class
mean_iou = tfk.metrics.MeanIoU(num_classes=num_classes, ignore_class=0, sparse_y_pred=False)

# Define Callbacks
callbacks = [
    EarlyStopping(monitor="val_mean_io_u", patience=10, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.5, patience=5, min_lr=1e-6)
]

In [36]:
# Calculate class weights
unique_classes, pixel_counts = np.unique(y_train, return_counts=True)
class_weights = {cls: 1.0 / count for cls, count in zip(unique_classes, pixel_counts)}
total_weight = sum(class_weights.values())
class_weights = [weight / total_weight for cls, weight in class_weights.items()]

# Possible losses
def dice_loss(y_true, y_pred):
    smooth = 1e-5

    y_true_onehot = tf.one_hot(tf.cast(y_true, tf.int32), depth=len(class_weights), axis=-1)
    y_true_onehot = tf.reshape(y_true_onehot, (-1, len(class_weights)))
    y_pred = tf.reshape(y_pred, (-1, len(class_weights)))

    num_samples = tf.shape(y_pred)[0]
    tiled_class_weights = tf.tile(tf.expand_dims(class_weights, axis=0), [num_samples, 1])

    intersection = K.sum(tiled_class_weights * y_true_onehot * y_pred, axis=-1)

    return 1 - (2. * intersection + smooth) / (K.sum(tiled_class_weights * y_true_onehot, axis=-1) + K.sum(y_pred, axis=-1) + smooth)


def focal_loss(y_true, y_pred):

    y_true_one_hot = tf.one_hot(tf.cast(y_true, tf.int32), depth=tf.shape(y_pred)[-1])
    y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)

    cross_entropy = -y_true_one_hot * tf.math.log(y_pred)
    focal_term = tf.pow(1 - y_pred, 2.0)
    loss = tf.reduce_sum(focal_term * cross_entropy, axis=-1)

    return tf.reduce_mean(loss)

def composite_loss(y_true, y_pred):
    ce_loss = tf.keras.losses.SparseCategoricalCrossentropy()(y_true, y_pred)
    dl_loss = dice_loss(y_true, y_pred)
    fl_loss = focal_loss(y_true, y_pred)

    max_ce = 1.3  # Max value observed for cross-entropy
    max_dice = 1.0  # Max value observed for dice
    max_focal = 0.7  # Max value observed for focal

    # Normalize losses
    ce_loss = ce_loss / max_ce
    dl_loss = dl_loss / max_dice
    fl_loss = fl_loss / max_focal

    return dl_loss

In [37]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=composite_loss,
              metrics=[mean_iou])

In [38]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    batch_size=32,
    callbacks=callbacks
)

Epoch 1/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 103s 838ms/step - loss: 0.9961 - mean_io_u: 0.1414 - val_loss: 0.9971 - val_mean_io_u: 0.0683 - learning_rate: 0.0010
Epoch 2/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 64s 224ms/step - loss: 0.9952 - mean_io_u: 0.1675 - val_loss: 0.9971 - val_mean_io_u: 0.0683 - learning_rate: 0.0010
Epoch 3/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 20s 223ms/step - loss: 0.9950 - mean_io_u: 0.1820 - val_loss: 0.9971 - val_mean_io_u: 0.0683 - learning_rate: 0.0010
Epoch 4/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 21s 227ms/step - loss: 0.9947 - mean_io_u: 0.2299 - val_loss: 0.9971 - val_mean_io_u: 0.0549 - learning_rate: 0.0010
Epoch 5/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 20s 215ms/step - loss: 0.9942 - mean_io_u: 0.2896 - val_loss: 0.9971 - val_mean_io_u: 0.0550 - learning_rate: 0.0010
Epoch 6/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 21s 219ms/step - loss: 0.9939 - mean_io_u: 0.3119 - val_loss: 0.9966 - val_mean_io_u: 0.1030 - learning_rate: 0.0010
Epoch 7/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 15s 224ms/step - loss: 0

In [39]:
timestep_str = datetime.now().strftime("%y%m%d_%H%M%S")
model_filename = f"model_{timestep_str}.keras"
model.save(model_filename)
del model

print(f"Model saved to {model_filename}")

Model saved to model_241203_081328.keras


## 📊 Prepare Your Submission

In our Kaggle competition, submissions are made as `csv` files. To create a proper `csv` file, you need to flatten your predictions and include an `id` column as the first column of your dataframe. To maintain consistency between your results and our solution, please avoid shuffling the test set. The code below demonstrates how to prepare the `csv` file from your model predictions.




In [40]:
# If model_filename is not defined, load the most recent model from Google Drive
if "model_filename" not in globals() or model_filename is None:
    files = [f for f in os.listdir('.') if os.path.isfile(f) and f.startswith('model_') and f.endswith('.keras')]
    files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
    if files:
        model_filename = files[0]
    else:
        raise FileNotFoundError("No model files found in the current directory.")

In [41]:
model = tfk.models.load_model(model_filename, custom_objects={'composite_loss': composite_loss})
print(f"Model loaded from {model_filename}")

Model loaded from model_241203_081328.keras


In [42]:
preds = model.predict(X_test)
preds = np.argmax(preds, axis=-1)
print(f"Predictions shape: {preds.shape}")

314/314 ━━━━━━━━━━━━━━━━━━━━ 27s 84ms/step
Predictions shape: (10022, 64, 128)


In [43]:
def y_to_df(y) -> pd.DataFrame:
    """Converts segmentation predictions into a DataFrame format for Kaggle."""
    n_samples = len(y)
    y_flat = y.reshape(n_samples, -1)
    df = pd.DataFrame(y_flat)
    df["id"] = np.arange(n_samples)
    cols = ["id"] + [col for col in df.columns if col != "id"]
    return df[cols]

In [44]:
# Create and download the csv submission file
timestep_str = model_filename.replace("model_", "").replace(".keras", "")
submission_filename = f"submission_{timestep_str}.csv"
submission_df = y_to_df(preds)
submission_df.to_csv(submission_filename, index=False)

from google.colab import files
files.download(submission_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#  
<img src="https://airlab.deib.polimi.it/wp-content/uploads/2019/07/airlab-logo-new_cropped.png" width="350">

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/95/Instagram_logo_2022.svg/800px-Instagram_logo_2022.svg.png" width="15"> **Instagram:** https://www.instagram.com/airlab_polimi/

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/LinkedIn_icon.svg/2048px-LinkedIn_icon.svg.png" width="15"> **LinkedIn:** https://www.linkedin.com/company/airlab-polimi/
___
Credits: Alberto Archetti 📧 alberto.archetti@polito.it





```
   Copyright 2024 Alberto Archetti

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
```